## Previo
- Agarrar los datos del tp anterior
- Bajarse el dataset de testing de Kaggle (no usado actualmente)

In [5]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

## Inicializo el DF

In [4]:
dtypes = {
    'country_name': str,
    'expenses': object,
    'id': str,
    'title': str,
    'state_name': str,
    'lat': float,
    'lon': float
}

cols = range(6) + range(8, 18) + [20, 21] + range(23, 27)
parse_dates = ['created_on']

df = pd.read_csv('data_gba_total.csv', dtype=dtypes, usecols=cols, parse_dates=parse_dates)
df = df.select_dtypes(['number'])
df.sample(5)

,floor,geonames_id,lat,lon,price,price_aprox_local_currency,price_aprox_usd,price_per_m2,rooms,surface_covered_in_m2,surface_in_m2,surface_total_in_m2
196585,NaN,3436134.0,-34.641094,-58.377395,92000.0,1469102.00,92000.0,1642.857143,3.0,56.0,NaN,66.0
1281206,NaN,3430234.0,-34.584782,-58.416664,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN
846797,10.0,NaN,-34.605372,-58.386505,156000.0,898281.17,156000.0,NaN,3.0,NaN,55.0,NaN
347087,NaN,NaN,-34.409530,-58.614391,386000.0,6163841.00,386000.0,3574.074074,3.0,108.0,NaN,108.0
2303658,2.0,3862356.0,NaN,NaN,71500.0,645973.90,71500.0,NaN,2.0,NaN,40.0,NaN


### Cleanup

Cleanup muy malo a mejorar, hay que usar mas columnas, actualmente descarto demasiada informacion para poder tener un dataset limpio usable mas rapido

In [7]:
df = df.fillna(value=0)
df = df.loc[df['price_aprox_usd'] > 5000]
price = df['price_aprox_usd']
for column in df.columns:
    if 'price' in column:
        del df[column]
        continue


del df['surface_in_m2']  # Existen otras columnas, surface_total_in_m2
del df['geonames_id']

In [8]:
df.shape

(2085663, 6)

In [9]:
# Parto el dataframe en train y test
x_train, x_test, y_train, y_test = train_test_split(df, price)

Busco el mejor hiperparametro, haciendo grid search. En el caso de Random Forest el unico hiper parametro es la cantidad de features (columnas) a usar por arbol, entonces itero los valores posibles (de 1 a 6) y me quedo con el que minimice el error.

In [34]:
features = len(df.columns)

best = -1
best_error = 99999999999999999999999999
for i in range(1, features):

    regressor = RandomForestRegressor(max_features=i)
    regressor.fit(x_train, y_train)
    result = regressor.predict(x_test)
    error = mean_squared_error(result, y_test)
    if error < best_error:
        best = i
        best_error = error

best

2

In [37]:
error  # Esperado: 11 cifras, parece ser normal y esperado (por el momento)

45684972498.908867